In [1]:
%load_ext sql

In [2]:
%sql postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics

In [6]:
%%sql SELECT subquery_result.sport, AVG(subquery_result.age) AS average_age
  FROM (
    SELECT 
      name,
      sport,
      DATE_PART('year', CURRENT_DATE) - DATE_PART('year', date_of_birth) as age
      FROM athletes
  ) subquery_result
  GROUP BY subquery_result.sport
  ORDER BY average_age DESC
  LIMIT 5

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
5 rows affected.


sport,average_age
Shooting,42.86363636363637
Equestrian,41.506493506493506
Cycling Road,40.798122065727696
Archery,40.75539568345324
Powerlifting,39.146067415730336


In [4]:
%%sql SELECT subquery_result.medal, AVG(subquery_result.age) as average_age
FROM (
  SELECT
    medals.medal,
    DATE_PART('year', CURRENT_DATE) - DATE_PART('year', athletes.date_of_birth) as age
    FROM athletes INNER JOIN medals
    ON athletes.name = medals.name
    AND athletes.sport = medals.sport
) subquery_result
GROUP BY medal
ORDER BY average_age DESC

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


medal,average_age
Bronze Medal,34.05710491367862
Silver Medal,33.65123010130246
Gold Medal,32.70487106017192


In [7]:
%%sql SELECT name
FROM athletes
WHERE name IN (
    SELECT name
    FROM medals
    WHERE medal = 'Gold Medal'
)
LIMIT 5

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
5 rows affected.


name
ABARZA Alberto
ABDELLAOUI Cherine
ABLINGER Walter
ABRAHAM GEBRU Daniel
ABRAHAM GEBRU Daniel


In [9]:
%%sql SELECT name
FROM athletes
WHERE name IN (
    SELECT name
    FROM athletes
    GROUP BY name
    HAVING COUNT(DISTINCT sport) > 1
)
LIMIT 5

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
5 rows affected.


name
ABRAHAM GEBRU Daniel
ABRAHAM GEBRU Daniel
ASTASHOV Mikhail
ASTASHOV Mikhail
BANGMA Tristan


In [11]:
%%sql SELECT date_of_birth, athletes.name, athletes.sport, gold_medals.medal
FROM athletes
JOIN (
    SELECT name, event, medal
    FROM medals
    WHERE medal = 'Gold Medal'
) AS gold_medals ON athletes.name = gold_medals.name
LIMIT 5

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
5 rows affected.


date_of_birth,name,sport,medal
1984-12-11,ABARZA Alberto,Swimming,Gold Medal
1998-08-28,ABDELLAOUI Cherine,Judo,Gold Medal
1969-05-12,ABLINGER Walter,Cycling Road,Gold Medal
1985-02-11,ABRAHAM GEBRU Daniel,Cycling Road,Gold Medal
1985-02-11,ABRAHAM GEBRU Daniel,Cycling Track,Gold Medal


In [12]:
%sql SELECT * FROM medals LIMIT 3

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


index,country,countrycode,name,sport,sport_short,event,medal
0,Czech Republic,CZE,DRAHONINSKY David,Archery,ARC,Men's Individual - W1,Gold Medal
1,People's Republic of China,CHN,HE Zihao,Archery,ARC,Men's Individual Compound - Open,Gold Medal
2,United States of America,USA,MATHER Kevin,Archery,ARC,Men's Individual Recurve - Open,Gold Medal


In [13]:
%sql SELECT * FROM athletes LIMIT 3

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


index,name,country,countrycode,sport,sport_code,date_of_birth,gender
0,AAJIM Munkhbat,Mongolia,MGL,Judo,JUD,1989-01-25,Male
1,ABARZA Alberto,Chile,CHI,Swimming,SWM,1984-12-11,Male
2,ABASLI Namig,Azerbaijan,AZE,Judo,JUD,1997-09-12,Male


In [47]:
%%sql SELECT country, 
      COUNT(medal)
      FROM medals
      GROUP BY country
      ORDER BY count DESC
      LIMIT 2

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
2 rows affected.


country,count
People's Republic of China,276
Great Britain,171


In [46]:
%%sql SELECT country, 
      COUNT(name)
      FROM athletes
      GROUP BY country
      ORDER BY count DESC
      LIMIT 2

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
2 rows affected.


country,count
Japan,262
People's Republic of China,256


In [88]:
%%sql 
SELECT 
    ath_count.country, 
    ath_count.athlete_count, 
    medal_count.medal_count,
ROUND(
    CAST(medal_count.medal_count AS numeric) / ath_count.athlete_count, 2) AS medals_per_athlete
FROM(SELECT country, COUNT(name) AS athlete_count FROM athletes GROUP BY country) AS ath_count
JOIN(SELECT country, COUNT(medal) AS medal_count FROM medals GROUP BY country) AS medal_count
ON ath_count.country = medal_count.country
ORDER BY medals_per_athlete DESC
LIMIT 5


 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
5 rows affected.


country,athlete_count,medal_count,medals_per_athlete
People's Republic of China,256,276,1.08
Netherlands,74,77,1.04
Italy,114,91,0.80
Ukraine,139,108,0.78
Great Britain,221,171,0.77
